<a href="https://colab.research.google.com/github/Mruduldhawley/netflix_recommender_system/blob/master/Netflix_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Netflix recomendation system

In [20]:
import pandas as pd
import numpy as np


In [5]:
from google.colab import files
uploaded=files.upload()

Saving netflix_titles.csv to netflix_titles.csv


In [21]:
import matplotlib.pyplot as plt
import seaborn as snb
%matplotlib inline

In [22]:
df = pd.read_csv('netflix_titles.csv')

In [23]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [24]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [26]:
df['director'].fillna('missing',inplace=True)

In [27]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [28]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,missing,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [29]:
df[df['rating'].isna()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
67,s68,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,missing,"Oprah Winfrey, Ava DuVernay",NaN,"January 26, 2017",2017,NaN,37 min,Movies,Oprah Winfrey sits down with director Ava DuVe...
2359,s2360,TV Show,Gargantia on the Verdurous Planet,missing,"Kaito Ishikawa, Hisako Kanemoto, Ai Kayano, Ka...",Japan,"December 1, 2016",2013,NaN,1 Season,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel..."
3660,s3661,TV Show,Little Lunch,missing,"Flynn Curry, Olivia Deeble, Madison Lu, Oisín ...",Australia,"February 1, 2018",2015,NaN,1 Season,"Kids' TV, TV Comedies","Adopting a child's perspective, this show take..."
3736,s3737,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,NaN,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi..."
3737,s3738,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,NaN,84 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...
3738,s3739,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,NaN,66 min,Movies,The comic puts his trademark hilarious/thought...
4323,s4324,Movie,My Honor Was Loyalty,Alessandro Pepe,"Leone Frisa, Paolo Vaccarino, Francesco Miglio...",Italy,"March 1, 2017",2015,NaN,115 min,Dramas,"Amid the chaos and horror of World War II, a c..."


In [30]:
df['rating'].value_counts()

TV-MA       2863
TV-14       1931
TV-PG        806
R            665
PG-13        386
TV-Y         280
TV-Y7        271
PG           247
TV-G         194
NR            84
G             39
TV-Y7-FV       6
UR             5
NC-17          3
Name: rating, dtype: int64

In [31]:
replace_nan={67:'TV-PG',
             2359:'PG-13',
             3660:'PG',
             3736:'PG-13',
             3737:'PG-13',
             3738:'G',
             4323:'PG-13'}

for id, rate in replace_nan.items():
    df.iloc[id, 8] = rate             

In [34]:
df.isnull().sum()

show_id           0
type              0
title             0
director          0
cast            718
country         507
date_added       10
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

In [37]:
frequent=df['country'].mode()[0]

In [39]:
df['country'].fillna(frequent,inplace=True)

In [42]:
df['cast'].fillna('missing',inplace=True)

In [52]:
df['date_added'].fillna('',inplace=True)

In [55]:
df['cast'] = df['cast'].apply(lambda x : x.replace(',', ''))
df['director'] = df['director'].apply(lambda x : x.replace(',', ''))
df['listed_in'] = df['listed_in'].apply(lambda x : x.replace(',',''))
df['listed_in'] = df['listed_in'].apply(lambda x : x.replace('&',''))

In [58]:
df['total']=df['type']+df['director']+df['cast']+df['country']+df['duration']+df['listed_in']+df['rating']+df['description']

In [59]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,total
0,s1,TV Show,3%,missing,João Miguel Bianca Comparato Michel Gomes Rodo...,Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,International TV Shows TV Dramas TV Sci-Fi Fa...,In a future where the elite inhabit an island ...,TV ShowmissingJoão Miguel Bianca Comparato Mic...
1,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir Héctor Bonilla Oscar Serrano Aza...,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas International Movies,After a devastating earthquake hits Mexico Cit...,MovieJorge Michel GrauDemián Bichir Héctor Bon...
2,s3,Movie,23:59,Gilbert Chan,Tedd Chan Stella Chung Henley Hii Lawrence Koh...,Singapore,"December 20, 2018",2011,R,78 min,Horror Movies International Movies,"When an army recruit is found dead, his fellow...",MovieGilbert ChanTedd Chan Stella Chung Henley...
3,s4,Movie,9,Shane Acker,Elijah Wood John C. Reilly Jennifer Connelly C...,United States,"November 16, 2017",2009,PG-13,80 min,Action Adventure Independent Movies Sci-Fi F...,"In a postapocalyptic world, rag-doll robots hi...",MovieShane AckerElijah Wood John C. Reilly Jen...
4,s5,Movie,21,Robert Luketic,Jim Sturgess Kevin Spacey Kate Bosworth Aaron ...,United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...,MovieRobert LuketicJim Sturgess Kevin Spacey K...


In [63]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from nltk.tokenize import word_tokenize

In [64]:
tfv = CountVectorizer()

In [65]:
text_feat=tfv.fit_transform(df.total)

In [82]:
similarity_matrix=cosine_similarity(text_feat)

In [69]:
import random 

In [91]:
new=random.sample(list(df.title),1)

In [92]:
new[0]

'Judah Friedlander: America Is the Greatest Country in the United States'

In [103]:
def movrec (movie_title ):
    movie_index = df[df.title == movie_title].index
    similarity_score = similarity_matrix[movie_index]
    movie_frame = pd.DataFrame(similarity_score[0],columns=['cosine_similarity'])
    movie_frame=movie_frame.reset_index()
    movie_frame = movie_frame.sort_values(by='cosine_similarity',ascending=False)
    top10 = 10 
    top10_index = list(movie_frame['index'])[:10]
    recommended_movies = {}
    for i in top10_index:
        temp = df['title'][i]
        score = movie_frame['cosine_similarity'][i]
        recommended_movies[temp]=score
    for i in recommended_movies.items():
      print(i[0])


In [117]:
import random
movie_title = random.sample(list(df['title']),1)
print('Movies like movie ->' ,movie_title[0], 'are :\n\n')
movrec(movie_title[0])

Movies like movie -> A Very Murray Christmas are :


A Very Murray Christmas
Life in a ... Metro
ReMastered: Devil at the Crossroads
Chris Brown: Welcome to My Life
Listen Up! The Lives of Quincy Jones
Behind the Curtain: Todrick Hall
What We Started
John Mellencamp: Plain Spoken
The Shaukeens
Mariah Carey's Merriest Christmas
